### Activity: Create interactive temporal visualization  

In this activity we will combine all the knowledge of time-series manipulation and range-slider learned in
previous exercises. 
Interactive slider is commnly used in time-series visualization

**import the required libraries**

In [1]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from pathlib import Path
import pandas as pd
import numpy as np
from ipywidgets import interact
%matplotlib inline

**setup the output to jupyter notebook**

In [2]:
DATA_PATH = Path("../datasets/chap5_data/")
output_notebook()

Loading BokehJS ...

**create a dataframe `microsoft_df` and parse the date column**

In [3]:
microsoft_df = pd.read_csv(DATA_PATH / "microsoft_stock.csv", parse_dates=['date'])
microsoft_df.head()

,date,open,high,low,close,volume,adj_close
0,2000-03-01,89.62,94.09,88.94,90.81,106889800,33.68
1,2000-03-02,91.81,95.37,91.12,93.37,106932600,34.63
2,2000-03-03,94.75,98.87,93.87,96.12,101435200,35.65
3,2000-03-06,96.00,97.37,90.12,90.62,93609400,33.61
4,2000-03-07,96.12,97.50,91.94,92.87,135061000,34.45


**Set the index as date**

In [4]:
microsoft_df.index = microsoft_df.date

**create `date` numpy array and `source` as ColumnDataSource. We will use these to draw line plot.**

In [5]:
dates = np.array(microsoft_df['date'], dtype=np.datetime64)
source = ColumnDataSource(data=dict(date=dates, high=microsoft_df['high']))

**initialize the figure and drwa the line**

In [6]:
p = figure(title="Time Series Stock Data for Microsoft", plot_height=300, plot_width=800,
           x_range=(dates[1000], dates[1800]), x_axis_location="above", x_axis_type="datetime", 
           tools="xpan", toolbar_location=None,
           background_fill_color="#ffefef")

p.yaxis.axis_label = 'High Price for Day'
r = p.line('date', 'high', source=source)

**Create range slider using `RangeTool`**

In [7]:
select = figure(title="Drag To See More Data",plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None, plot_height=130,
                tools="", background_fill_color="#ffefef", toolbar_location=None,)

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "green"
range_tool.overlay.fill_alpha = 0.2

**Write a custom update function which aggregate data by `month`, `year` and `day`**

In [8]:
def update(f):
    if   f == "day":
        r.data_source.data = dict({
            'date': microsoft_df.index,
            'high': microsoft_df.high
        })
    elif f == "month":
        month = microsoft_df.groupby(pd.Grouper(freq="M"))[['high']].mean()
        r.data_source.data = dict({
            'date': month.index,
            'high': month.high
        })
    elif f == "year":
        year = microsoft_df.groupby(pd.Grouper(freq="Y"))[['high']].mean()
        r.data_source.data = dict({
            'date': year.index,
            'high': year.high
        })
    
    push_notebook()
    
select.line('date', 'high', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool
show(column(p, select), notebook_handle=True)

**Plot both range slider and aggregator on figure**

In [9]:
select.line('date', 'high', source=source)
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool
show(column(p, select), notebook_handle=True)
interact(update, f=["day", "month", "year"])

interactive(children=(Dropdown(description='f', options=('day', 'month', 'year'), value='day'), Output()), _do…

<function __main__.update(f)>